<a href="https://colab.research.google.com/github/AlexLZM/ML-lab-Final-Project/blob/main/ML_lab_Final_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.base import BaseEstimator
from sklearn.compose import *
from sklearn.experimental import enable_hist_gradient_boosting, enable_iterative_imputer
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    HistGradientBoostingClassifier,
    RandomForestClassifier,
    StackingClassifier,
    VotingClassifier,
)
from sklearn.impute import *
from sklearn.inspection import permutation_importance
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import *
!pip install optuna
import optuna

class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass

     |████████████████████████████████| 296kB 6.1MB/s 
     |████████████████████████████████| 163kB 9.6MB/s 
     |████████████████████████████████| 81kB 5.4MB/s 
     |████████████████████████████████| 81kB 3.9MB/s 
     |████████████████████████████████| 112kB 10.9MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 143kB 10.6MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=73aef75ae178fdc9da0dee82f2b0b9a1dcab9e1273c48cca9cdb1efb5799d5f3
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


# MSDS 699 Final Project



## Modeling for Hear Disease Prediction




## Student: Zhimin Lyu

## Project description

This dataset gives information of patients related to heart disease. Dataset contains 14 features of patients' conditons and test results, target variable is the fact that the patient has heart disease or not. 

In the project, we set the aim to build a decent model to predict the target variable (disease\non disease) by comparing different machine learning algorithms.

<b>Data Attribute Information</b>

From (Area the patient came from)

Age (age in years)

Sex (1 = male; 0 = female)

CP (chest pain type)

TRESTBPS (resting blood pressure (in mm Hg on admission to the hospital))

CHOL (serum cholestoral in mg/dl)

FPS (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)

RESTECH (resting electrocardiographic results)

THALACH (maximum heart rate achieved)

EXANG (exercise induced angina (1 = yes; 0 = no))

OLDPEAK (ST depression induced by exercise relative to rest)

SLOPE (the slope of the peak exercise ST segment)

CA (number of major vessels (0-3) colored by flourosopy)

THAL (3 = normal; 6 = fixed defect; 7 = reversable defect)

Disease (has heart disease or not: 1 or 0)

## Load Data

In [ ]:
data = pd.read_csv(
    'https://github.com/AlexLZM/Testit/raw/main/Heart_Disease%20(1).csv')

In [ ]:
# Data split
X, y = data.iloc[:, :-1], data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=666)

## Fit scikit-learn model

<b>Create pipeline for categorical and numeric values</b>

In [ ]:

# seperate categorical and numerical features based on domain knowlage
cate = [0, 2, 3, 7, 9, 11, 13]
nume = [1, 4, 5, 6, 8, 9, 12]

con_pipe = Pipeline([('sca', StandardScaler()),  # apply standardscaler to numeric features
                     ('imputer', SimpleImputer(strategy='median'))])  # fill null value with median
cat_pipe = Pipeline([('str',
                      FunctionTransformer(lambda x:(x.astype(str)),
                                          validate=False)),  # transform integer categoricals in to string
                     # fill null value with 'unknown'
                     ('imputer', SimpleImputer(
                         strategy='constant', fill_value='unknown')),
                     ('encoder', OneHotEncoder(handle_unknown='ignore'))])  # apply one-hot encoder to categoricals
prep = ColumnTransformer(
    [('conti', con_pipe, nume), ('cate', cat_pipe, cate)])
# Dummy for Randomized search
pipe = Pipeline([('prep', prep), ('clf', DummyEstimator())])

In [ ]:
# Randomized search withing hyperparameters and model families
search_space = [{'clf': [RandomForestClassifier(n_jobs=-1)],  # Estimator 1
                 # controls dependence of trees
                 'clf__max_features': np.arange(0.1, 0.6, 0.1),
                 # should be enough to converge to minimum validation loss
                 'clf__n_estimators': np.arange(100, 1000, 100),
                 # control tree size to reduce overfitting
                 'clf__min_samples_leaf':np.arange(1, 22, 5),
                 # control tree size to reduce overfitting
                 'clf__max_depth': np.arange(1, 22, 2),
                 # control tree size to reduce overfitting
                 'clf__max_leaf_nodes': np.arange(10, 200, 20),
                 # bootstrap/subsampling choice
                 'clf__bootstrap': [True, False],
                 # control tree size to reduce overfitting
                 'clf__min_samples_split': np.arange(2, 22, 2)
                 },

                {'clf': [GradientBoostingClassifier()],  # Estimator 2
                 # combine with learning rate to reduce overfitting
                 'clf__n_estimators': np.arange(100, 1000, 100),
                 # control tree size to reduce overfitting
                 'clf__min_samples_leaf':np.arange(1, 22, 5),
                 # control tree size to reduce overfitting
                 'clf__max_depth': np.arange(1, 22, 2),
                 # lower lr means slower fitting process
                 'clf__learning_rate':np.arange(0.1, 1.1, 0.1),
                 # control tree size to reduce overfitting
                 'clf__max_leaf_nodes': np.arange(10, 200, 20),
                 # control tree size to reduce overfitting
                 'clf__min_samples_split': np.arange(2, 22, 2)
                 },
                {'clf': [AdaBoostClassifier()],  # Estimator 3
                 # combine with learning rate to reduce overfitting
                 'clf__n_estimators': np.arange(100, 1000, 100),
                 # lower lr means slower fitting process
                 'clf__learning_rate':np.arange(0.1, 1.1, 0.1),
                 },
                {'clf': [HistGradientBoostingClassifier()],  # Estimator 4
                 # control tree size to reduce overfitting
                 'clf__min_samples_leaf':np.arange(1, 22, 5),
                 # control tree size to reduce overfitting
                 'clf__max_depth': np.arange(1, 22, 2),
                 # control tree size to reduce overfitting
                 'clf__max_leaf_nodes': np.arange(10, 200, 20),
                 # lower lr means slower fitting process
                 'clf__learning_rate':np.arange(0.1, 1.1, 0.1),
                 # reduce overfit but increase bias
                 'clf__l2_regularization':np.logspace(0.00001, 10, 10)

                 }
                ]

# random search instance
rand_search = RandomizedSearchCV(estimator=pipe,
                                 param_distributions=search_space,
                                 n_iter=50,
                                 cv=5,
                                 n_jobs=-1,
                                 verbose=1, scoring='f1')


# Fit the search instance
rand_search.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  2.1min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('prep',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='drop',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('conti',
                                                                               Pipeline(memory=None,
                                                                                        steps=[('sca',
                                                                                                StandardScaler(copy=True,
                                                                                                        

In [ ]:
print(rand_search.best_params_, rand_search.best_score_)

{'clf__n_estimators': 900, 'clf__min_samples_split': 10, 'clf__min_samples_leaf': 1, 'clf__max_leaf_nodes': 70, 'clf__max_features': 0.1, 'clf__max_depth': 19, 'clf__bootstrap': True, 'clf': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=19, max_features=0.1,
                       max_leaf_nodes=70, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=900,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)} 0.8556411581994204


In [ ]:
# Baysian Optimization for Hyperparameters
# Caution!! Takes long time!


def objective(trial):

    # Invoke suggest methods of a Trial object to generate hyperparameters.
    classifier_names = trial.suggest_categorical(
        'reg', ['pipe1', 'pipe2', 'pipe3', 'pipe4'])
    if classifier_names == 'pipe1':
        n_esti = trial.suggest_int('n_esti', 100, 1000)
        msl = trial.suggest_int('msl', 1, 20)
        md = trial.suggest_int('md', 1, 20)
        mf = trial.suggest_uniform('mf', 0.1, 0.5)
        mln = trial.suggest_int('mln', 10, 200)
        bs = trial.suggest_categorical('bs', [True, False])
        mss = trial.suggest_int('mss', 2, 20)
        cw = trial.suggest_categorical(
            'cw', ['balanced', 'balanced_subsample', None])

        pipee = Pipeline([('prep', prep),
                          ('clf', RandomForestClassifier(max_features=mf,
                                                         n_jobs=-1,
                                                         n_estimators=n_esti,
                                                         min_samples_leaf=msl,
                                                         max_depth=md,
                                                         class_weight=cw,
                                                         max_leaf_nodes=mln,
                                                         bootstrap=bs,
                                                         min_samples_split=mss
                                                         ))])

    elif classifier_names == 'pipe2':
        n_esti = trial.suggest_int('n_esti', 100, 1000)

        learning_rate = trial.suggest_uniform('lr', 0.1, 1)
        pipee = Pipeline([('prep', prep), ('clf', AdaBoostClassifier(
            n_estimators=n_esti,
            learning_rate=learning_rate
        ))])
    elif classifier_names == 'pipe3':
        msl = trial.suggest_int('msl', 1, 20)
        md = trial.suggest_int('md', 1, 20)
        mln = trial.suggest_int('mln', 10, 200)
        learning_rate = trial.suggest_uniform('lr', 0.1, 1)
        l2_regularization = trial.suggest_loguniform('l2', 0.00001, 10)
        pipee = Pipeline([('prep', prep),
                          ('clf', HistGradientBoostingClassifier(
                              min_samples_leaf=msl,
                              max_depth=md,
                              max_leaf_nodes=mln,
                              learning_rate=learning_rate,
                              l2_regularization=l2_regularization,
                              ))])
    else:
        n_esti = trial.suggest_int('n_esti', 100, 1000)
        msl = trial.suggest_int('msl', 1, 20)
        md = trial.suggest_int('md', 1, 20)
        mln = trial.suggest_int('mln', 10, 200)
        mss = trial.suggest_int('mss', 2, 20)
        learning_rate = trial.suggest_uniform('lr', 0.1, 1)
        pipee = Pipeline([('prep', prep),
                          ('clf', GradientBoostingClassifier(
                              n_estimators=n_esti,
                              min_samples_leaf=msl,
                              max_depth=md,
                              max_leaf_nodes=mln,
                              learning_rate=learning_rate,
                              min_samples_split=mss
                              ))])

    # choose f1 score as it is suitable for both imbalanced data and balanced data
    scores = cross_validate(pipee, X_train, y_train,
                            scoring='f1', cv=5, n_jobs=-1)

    error = -np.mean(scores['test_score'])

    return error  # An objective value linked with the Trial object.


study = optuna.create_study()  # Create a new study.
study.optimize(objective, n_trials=50, n_jobs=1, show_progress_bar=1)
print(study.best_params)
study.trials_dataframe().sort_values('value').head(10)

[I 2021-03-10 08:41:31,984] A new study created in memory with name: no-name-c4ab7f73-605e-4932-9e8d-d9383ec3c94b
/usr/local/lib/python3.7/dist-packages/optuna/progress_bar.py:47: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



[I 2021-03-10 08:41:33,031] Trial 0 finished with value: -0.8245283138984583 and parameters: {'reg': 'pipe4', 'n_esti': 240, 'msl': 7, 'md': 6, 'mln': 92, 'mss': 2, 'lr': 0.6767656595105165}. Best is trial 0 with value: -0.8245283138984583.
[I 2021-03-10 08:41:34,529] Trial 1 finished with value: -0.820152646839631 and parameters: {'reg': 'pipe1', 'n_esti': 149, 'msl': 18, 'md': 13, 'mf': 0.3182473698304479, 'mln': 158, 'bs': False, 'mss': 12, 'cw': 'balanced'}. Best is trial 0 with value: -0.8245283138984583.
[I 2021-03-10 08:41:41,905] Trial 2 finished with value: -0.8293893839448332 and parameters: {'reg': 'pipe1', 'n_esti': 830, 'msl': 9, 'md': 15, 'mf': 0.4099360760514724, 'mln': 151, 'bs': True, 'mss': 6, 'cw': 'balanced'}. Best is trial 2 with value: -0.8293893839448332.
[I 2021-03-10 08:41:44,158] Trial 3 finished with value: -0.8332525863707734 and parameters: {'reg': 'pipe2', 'n_esti': 272, 'lr': 0.5797781744229226}. Best is trial 3 with value: -0.8332525863707734.
[I 2021-03

,number,value,datetime_start,datetime_complete,duration,params_bs,params_cw,params_l2,params_lr,params_md,params_mf,params_mln,params_msl,params_mss,params_n_esti,params_reg,state
23,23,-0.847296,2021-03-10 08:42:44.429173,2021-03-10 08:42:47.010955,0 days 00:00:02.581782,NaN,NaN,NaN,0.113621,NaN,NaN,NaN,NaN,NaN,315.0,pipe2,COMPLETE
11,11,-0.845829,2021-03-10 08:42:09.183387,2021-03-10 08:42:12.728715,0 days 00:00:03.545328,NaN,NaN,NaN,0.109302,NaN,NaN,NaN,NaN,NaN,432.0,pipe2,COMPLETE
43,43,-0.844690,2021-03-10 08:43:41.831139,2021-03-10 08:43:44.664118,0 days 00:00:02.832979,NaN,NaN,NaN,0.102818,NaN,NaN,NaN,NaN,NaN,345.0,pipe2,COMPLETE
24,24,-0.844677,2021-03-10 08:42:47.022683,2021-03-10 08:42:49.925734,0 days 00:00:02.903051,NaN,NaN,NaN,0.195555,NaN,NaN,NaN,NaN,NaN,353.0,pipe2,COMPLETE
46,46,-0.844544,2021-03-10 08:43:49.340259,2021-03-10 08:43:52.976715,0 days 00:00:03.636456,NaN,NaN,NaN,0.109078,NaN,NaN,NaN,NaN,NaN,447.0,pipe2,COMPLETE
47,47,-0.844318,2021-03-10 08:43:52.985927,2021-03-10 08:43:56.825144,0 days 00:00:03.839217,NaN,NaN,NaN,0.152626,NaN,NaN,NaN,NaN,NaN,468.0,pipe2,COMPLETE
33,33,-0.844189,2021-03-10 08:43:08.668294,2021-03-10 08:43:11.856981,0 days 00:00:03.188687,NaN,NaN,NaN,0.102046,NaN,NaN,NaN,NaN,NaN,390.0,pipe2,COMPLETE
21,21,-0.844126,2021-03-10 08:42:38.783111,2021-03-10 08:42:41.710447,0 days 00:00:02.927336,NaN,NaN,NaN,0.119122,NaN,NaN,NaN,NaN,NaN,352.0,pipe2,COMPLETE
44,44,-0.844108,2021-03-10 08:43:44.669955,2021-03-10 08:43:47.310125,0 days 00:00:02.640170,NaN,NaN,NaN,0.151679,NaN,NaN,NaN,NaN,NaN,317.0,pipe2,COMPLETE
12,12,-0.843622,2021-03-10 08:42:12.739815,2021-03-10 08:42:17.200329,0 days 00:00:04.460514,NaN,NaN,NaN,0.101296,NaN,NaN,NaN,NaN,NaN,540.0,pipe2,COMPLETE


In [ ]:
# Collected best hyperparameters with some hand tuning
params0 = dict(bootstrap=False, max_depth=19, max_features=0.1,
               max_leaf_nodes=170, min_samples_split=10,
               n_estimators=300, n_jobs=-1)

params1 = {'n_estimators': 100,
           'learning_rate': 0.129}

params2 = {'min_samples_leaf': 13,
           'max_depth': 2,
           'max_leaf_nodes': 45,
           'learning_rate': 0.27,
           'l2_regularization': 6.711291}

params3 = {'n_estimators': 130,
           'learning_rate': 0.161163}

params4 = {'n_estimators': 138,
           'min_samples_leaf': 12,
           'max_depth': 1,
           'max_leaf_nodes': 103,
           'min_samples_split': 19,
           'learning_rate': 0.5245992478750252}

params5 = {'n_estimators': 154,
           'min_samples_leaf': 11,
           'max_depth': 1,
           'max_leaf_nodes': 119,
           'min_samples_split': 10,
           'learning_rate': 0.5707539214903328}

params6 = {'n_estimators': 645,
           'min_samples_leaf': 2,
           'max_depth': 17,
           'max_features': 0.17,
           'max_leaf_nodes': 74,
           'bootstrap': False,
           'min_samples_split': 11,
           'class_weight': None}

In [ ]:
# Plus in hyperparameters into model pipelines

# Candidate from 1st Randomized search
pipe0 = Pipeline([('prep', prep),
                  ('clf',
                   RandomForestClassifier(**params0))])

# Candidate from 2nd Randomized search
pipe1 = Pipeline([('prep', prep),
                  ('clf',
                   AdaBoostClassifier(**params1))])

# Candidate from 1st Bayesian Optimization search

pipe2 = Pipeline([('prep', prep),
                  ('clf',
                   HistGradientBoostingClassifier(**params2))])

# Candidate from 2nd Bayesian Optimization search

pipe3 = Pipeline([('prep', prep),
                  ('clf',
                   AdaBoostClassifier(**params3))])

# Candidate from 3rd Bayesian Optimization search
pipe4 = Pipeline([('prep', prep),
                  ('clf',
                   GradientBoostingClassifier(**params4))])

# Candidate from 4th Bayesian Optimization search
pipe5 = Pipeline([('prep', prep),
                  ('clf',
                   GradientBoostingClassifier(**params5))])

# Candidate from 5th Bayesian Optimization search
pipe6 = Pipeline([('prep', prep),
                  ('clf',
                   RandomForestClassifier(**params6))])

In [ ]:
# compare performance with iterative cross-validation
# to minimize variance from randomness
scores = []
for i in range(7):  # number of candidates
    score = []
    for j in range(5):  # number of iterations for each candidate
        score.append(np.mean(cross_validate(globals()['pipe'+str(i)],
                                            X_train, y_train, scoring='f1',
                                            cv=5, n_jobs=-1)['test_score']))
    scores.append((np.mean(score), np.std(score),
                   'pipe' +
                   str(i), type(
                       (globals()['pipe'+str(i)]).steps[1][1]).__name__[:-10],
                   (globals()['params'+str(i)])))

In [ ]:
ranking = pd.DataFrame(scores,
                       columns=['mean_score', 'std', 'estimator', 'type', 'param'])\
    .sort_values('mean_score', ascending=False)\
    .reset_index(drop=True)\
    .sort_values('type')

In [ ]:
pd.options.display.max_colwidth = 200
ranking

,mean_score,std,estimator,type,param
5,0.838862,0.000000,pipe1,AdaBoost,"{'n_estimators': 100, 'learning_rate': 0.129}"
6,0.838738,0.000000,pipe3,AdaBoost,"{'n_estimators': 130, 'learning_rate': 0.161163}"
2,0.850211,0.000000,pipe4,GradientBoosting,"{'n_estimators': 138, 'min_samples_leaf': 12, 'max_depth': 1, 'max_leaf_nodes': 103, 'min_samples_split': 19, 'learning_rate': 0.5245992478750252}"
3,0.845639,0.000000,pipe5,GradientBoosting,"{'n_estimators': 154, 'min_samples_leaf': 11, 'max_depth': 1, 'max_leaf_nodes': 119, 'min_samples_split': 10, 'learning_rate': 0.5707539214903328}"
4,0.843937,0.000000,pipe2,HistGradientBoosting,"{'min_samples_leaf': 13, 'max_depth': 2, 'max_leaf_nodes': 45, 'learning_rate': 0.27, 'l2_regularization': 6.711291}"
0,0.853187,0.001457,pipe0,RandomForest,"{'bootstrap': False, 'max_depth': 19, 'max_features': 0.1, 'max_leaf_nodes': 170, 'min_samples_split': 10, 'n_estimators': 300, 'n_jobs': -1}"
1,0.851979,0.002066,pipe6,RandomForest,"{'n_estimators': 645, 'min_samples_leaf': 2, 'max_depth': 17, 'max_features': 0.17, 'max_leaf_nodes': 74, 'bootstrap': False, 'min_samples_split': 11, 'class_weight': None}"


In [ ]:
# Automaticlly select best models
best_rf = globals()[ranking[ranking['type'] == 'RandomForest'].iloc[0, 2]]
best_ada = globals()[ranking[ranking['type'] == 'AdaBoost'].iloc[0, 2]]
best_gbm = globals()[ranking[ranking['type']
                             == 'GradientBoosting'].iloc[0, 2]]
best_hist = globals()[ranking[ranking['type']
                              == 'HistGradientBoosting'].iloc[0, 2]]

In [ ]:
# Best in each family:Random Forest, GBM, Hist GBM, AdaBoost
estimators = [('p0', best_rf), ('p1', best_ada),
              ('p2', best_gbm), ('p3', best_hist)]

In [ ]:
# stacked ensemble
stack = StackingClassifier(estimators, cv=5, n_jobs=-1)

In [ ]:
# voting ensemble
vote = VotingClassifier(estimators, n_jobs=-1)

In [ ]:
# evaluate stacking and voting with cross validation
scores = []
for ensemble in ['stack', 'vote']:
    score = []
    for i in range(5):
        score.append(np.mean(cross_validate(globals()[ensemble], X_train, y_train,
                                            scoring='f1', cv=5, n_jobs=-1)
                             ['test_score']))
    scores.append((np.mean(score), np.std(score),
                   ensemble, type((globals()[ensemble])).__name__[:-10],
                   f'Ensemble of best Random Forest, GBM, Hist GBM and AdaBoost models'))

In [ ]:
ranking2 = pd.DataFrame(scores,
                        columns=['mean_score', 'std', 'estimator', 'type', 'param'])

In [ ]:
ranking_final = pd.concat([ranking, ranking2], axis=0)\
.sort_values('mean_score', ascending=False)\
            .reset_index(drop=True)\

In [ ]:
ranking_final

,mean_score,std,estimator,type,param
0,0.853187,0.001457,pipe0,RandomForest,"{'bootstrap': False, 'max_depth': 19, 'max_features': 0.1, 'max_leaf_nodes': 170, 'min_samples_split': 10, 'n_estimators': 300, 'n_jobs': -1}"
1,0.851979,0.002066,pipe6,RandomForest,"{'n_estimators': 645, 'min_samples_leaf': 2, 'max_depth': 17, 'max_features': 0.17, 'max_leaf_nodes': 74, 'bootstrap': False, 'min_samples_split': 11, 'class_weight': None}"
2,0.850211,0.000000,pipe4,GradientBoosting,"{'n_estimators': 138, 'min_samples_leaf': 12, 'max_depth': 1, 'max_leaf_nodes': 103, 'min_samples_split': 19, 'learning_rate': 0.5245992478750252}"
3,0.849371,0.002100,stack,Stacking,"Ensemble of best Random Forest, GBM, Hist GBM and AdaBoost models"
4,0.846199,0.000690,vote,Voting,"Ensemble of best Random Forest, GBM, Hist GBM and AdaBoost models"
5,0.845639,0.000000,pipe5,GradientBoosting,"{'n_estimators': 154, 'min_samples_leaf': 11, 'max_depth': 1, 'max_leaf_nodes': 119, 'min_samples_split': 10, 'learning_rate': 0.5707539214903328}"
6,0.843937,0.000000,pipe2,HistGradientBoosting,"{'min_samples_leaf': 13, 'max_depth': 2, 'max_leaf_nodes': 45, 'learning_rate': 0.27, 'l2_regularization': 6.711291}"
7,0.838862,0.000000,pipe1,AdaBoost,"{'n_estimators': 100, 'learning_rate': 0.129}"
8,0.838738,0.000000,pipe3,AdaBoost,"{'n_estimators': 130, 'learning_rate': 0.161163}"


In [ ]:
best_model = globals()[ranking_final.loc[0, 'estimator']]

<b>According to final scores,we chose this model below as our final model:</b>





In [ ]:
if type(best_model).__name__ == 'Pipeline':
    print(f'Final model is a {type(best_model.steps[1][1]).__name__}')
else:
    print(f'Final model is a {type(best_model).__name__}')
print()
print(f'With preprocessing pipeline:\n {prep}')
print()
print(f"With hyperparameters:\n{ranking_final.loc[0,'param']}")

Final model is a RandomForestClassifier

With preprocessing pipeline:
 ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('conti',
                                 Pipeline(memory=None,
                                          steps=[('sca',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True)),
                                                 ('imputer',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                   

In [ ]:
best_model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('prep',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('conti',
                                                  Pipeline(memory=None,
                                                           steps=[('sca',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True)),
                                                                  ('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
  

### Final evaluation on test set


In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
f1score = f1_score(y_test, y_pred)
print(f'F1 score for prediction on test set is {f1score:.4f}')

F1 score for prediction on test set is 0.8287


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy score for prediction on test set is {accuracy:.4f}')

Accuracy score for prediction on test set is 0.8130


In [ ]:
r = permutation_importance(
    best_model, X, y, scoring='f1', n_repeats=10, n_jobs=-1)

In [ ]:
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{X.columns[i]:<8}"
              f"{r.importances_mean[i]:.3f}"
              f" ± {r.importances_std[i]:.3f}")

cp      0.060 ± 0.007
exang   0.044 ± 0.004
slope   0.043 ± 0.007
thal    0.026 ± 0.004
from    0.024 ± 0.005
restecg 0.022 ± 0.003
chol    0.018 ± 0.003
sex     0.018 ± 0.003
age     0.017 ± 0.003
thalach 0.014 ± 0.003
trestbps0.012 ± 0.003
ca      0.010 ± 0.002
fbs     0.007 ± 0.001


### Conclusion

Randomized hyperparameter search and bayesian optimization search are very useful to find decent hyperparameters quickly.

Tree ensembles are effective in tabular data prediction as it has plenty of mearsures to reduce overfit. And thet worked as expected in this project.

The final model is decent to help doctors to diagnose if a patient has heart disease or not.
And it shows that CP (chest pain type) and EXANG (exercise induced angina (1 = yes; 0 = no)) are the first 2 important factors in the diagnosis.

### Next steps

Other ensembles such as Xgboost, catboost and lightgbm could be attempted in the future.